### 34.1 Metaclass

A metaclass is a class of a class. 
It allows you to customize class creation and behavior

What are the uses of metaclasses?
- automatically register a class
- validating class attributes and methods
- modify the class even before it is created
- enforce coding standards

### 34.2 Example

##### Creating a metaclass

In [19]:
class MyMeta(type):

    def __new__(cls, name, bases, dct):
        dct['class_id'] = name.upper()
        print('Creating a class: ')
        print(cls)
        print(name)
        print(bases)
        print(dct)
        return super().__new__(cls, name, bases, dct)
        

##### Using a metaclass

In [22]:
class MyClass(metaclass=MyMeta):

    def greet(self):
        return f"{self.class_id}"
    

Creating a class: 
<class '__main__.MyMeta'>
MyClass
()
{'__module__': '__main__', '__qualname__': 'MyClass', 'greet': <function MyClass.greet at 0x000001DCB4EB6B60>, 'class_id': 'MYCLASS'}


In [24]:
obj = MyClass()
obj.greet()

'MYCLASS'

What is __new__()?
Special method -static method
Creates a new instance of a class
Gets called before the __init__
Hence it can be used to customize the class creation itself
Usually used when you want to control how classes are created
Sometime you may want to inject some attributes, methods and validation logic in to the class

Let's try to use __new__ in a normal class

In [26]:
class MyClass:

    def __new__(cls):
        print("Creating the class...")
        instance = super().__new__(cls)
        return instance

    def __init__(self):
        print("Initialization starts...")

In [28]:
m = MyClass()

Creating the class...
Initialization starts...


### 34.3 Validating Attributes using Metaclasses

Let's say we want to make sure that the classes implement the required_attrs and it should be in the form of a list. In this case, we can create a metaclass that makes sure of this 

In [43]:
class ValidateAttrs(type):
    
    def __new__(cls, name, bases, dct):
        if 'required_attrs' not in dct:
            raise TypeError(f"Class {name} should implement required_attrs")
        if not isinstance(dct['required_attrs'], list):
            raise TypeError(f"required_attrs in Class {name} should be of type list")
        return super().__new__(cls, name, bases, dct)

In [51]:
# Implementing without required_attrs: it will not even allow your to create a class
class A(metaclass=ValidateAttrs):

    def __init__(self):
        print("Class created")

TypeError: Class A should implement required_attrs

In [47]:
# With required_attrs
class A(metaclass=ValidateAttrs):

    required_attrs = ["name", "email"]

    def __init__(self):
        print("Class created")

##### A much better class implementation with a purpose

In [66]:
class Person(metaclass=ValidateAttrs):

    required_attrs = ['name', 'email']

    def __init__(self, **kargs):

        missing = [attr for attr in Person.required_attrs if attr not in kargs]
        if missing:
            raise ValueError(f"Missing attributes: {missing}")

        for attr in Person.required_attrs:
            setattr(self, attr, kargs[attr])

In [68]:
p = Person(name="Anil")

ValueError: Missing attributes: ['email']

In [70]:
p = Person(name="Anil", email="anil@ust.com")

In [72]:
p